# 0_구글드라이브연동

In [0]:
import os
import re
import time
import cv2
import json
import pandas as pd
import numpy as np
import requests
import operator
import httplib2, urllib, base64
import matplotlib.pyplot as plt
%matplotlib inline 

In [0]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

# 0_저장 폴더생성

In [0]:
# 영상과 YOLO_log를 활용한 이미지 저장소 
!mkdir /content/gdrive/My\ Drive/darknet/data/result
!mkdir /content/gdrive/My\ Drive/darknet/data/result/face
!mkdir /content/gdrive/My\ Drive/darknet/data/result/person
!mkdir /content/gdrive/My\ Drive/darknet/data/result/person_half

In [0]:
# 3D로 변경예정 이미지 및 3D.obj 결과 저장소
!mkdir /content/gdrive/My\ Drive/darknet/data/3DFaceReconstruction/input
!mkdir /content/gdrive/My\ Drive/darknet/data/3DFaceReconstruction/output
!mkdir /content/gdrive/My\ Drive/darknet/data/3DFaceReconstruction/output/input

## 1_YOLO를 위한 가상환경

In [0]:
# 구글 코랩 가상환경
!apt-get update
!apt-get upgrade
!apt-get install build-essential
!apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev
!apt-get install libavcodec-dev libavformat-dev libswscale-d
!apt-get install libopencv-dev
!make
!ls -la "/content/gdrive/My Drive/darknet"
!sudo apt-get install tree
!tree /content/gdrive/My\ Drive/darknet/
!/usr/local/cuda/bin/nvcc --version
!tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.* -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
!git clone https://github.com/kriyeng/darknet/
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!apt install g++-5
!apt install gcc-5
!apt update
!apt upgrade
!git checkout feature/google-colab
!cmake .
!make
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet
!wget https://pjreddie.com/media/files/yolov3.weights

### 1-1 (PATH) YOLO

In [0]:
os.environ['PATH'] += ':/usr/local/cuda/bin'

!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

### 3-1 (PATH) Person profile

In [0]:
# face api에서 사용할 기능 path 설정
_url_detect = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect'
_url_verify = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/verify'
# face api 접속에 필요한 key값
_key = '3159124659a1473d8aebba9ff74fac84' 
_maxNumRetries = 10

### 3-2 (API) Person profile

In [0]:
# api 요청 코드
# 건들지 마시게!
def detectionRequest( json, data, headers, params ):

  """
  Helper function to process the request to Project Oxford

  Parameters:
  json: Used when processing images from its URL. See API Documentation
  data: Used when processing image read from disk. See API Documentation
  headers: Used to pass the key information and the data type request
  """

  retries = 0
  result = None

  while True:

    response = requests.request( 'post', _url_detect, json = json, data = data, headers = headers, params = params )

    if response.status_code == 429: 

      print( "Message: %s" % ( response.json()['error']['message'] ) )

      if retries <= _maxNumRetries: 
        time.sleep(1) 
        retries += 1
        continue
      else: 
        print( 'Error: failed after retrying!' )
        break

    elif response.status_code == 200 or response.status_code == 201:

      if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
        result = None 
      elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
        if 'application/json' in response.headers['content-type'].lower(): 
          result = response.json() if response.content else None 
        elif 'image' in response.headers['content-type'].lower(): 
          result = response.content
    else:
      print( "Error code: %d" % ( response.status_code ) )
      print( "Message: %s" % ( response.json()['error']['message'] ) )

    break
    
  return result

### 1-2 (함수) YOLO 출력, 업로드, 다운로드 

In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

### 2-1 (함수) Person 100% 추출

In [0]:
# 사람 정확도 뽑아내는 함수

def person_acc(data):
  if 'person: ' in data:
    return int(data.split('person: ')[-1].split('%')[0]) # 이해 안가면 여쭤봐주세요
  else: return 0

# fps 뽑아내는 함수
def fps(data):
  return float(data.split('FPS:')[-1])

def parsing_log(log_dir, accuracy):
  with open(log_dir, 'r') as f:
    string=f.read()
  
  p=re.compile(r'\n+')
  cleaned_string = p.sub('', string)
  
  cleaned_string = cleaned_string.replace('Stream closed. cvWriteFrame input video stream closed.  closing... closed!output_video_writer closed.', '')
  
  content = cleaned_string.split(' cvWriteFrame Objects:')[1:] # 반복되는 단어 기준으로 스트링을 쪼갭니다.
  # 0번이 아니라 1번부터 부르는 이유는 0번에 필요없는 문자가 들어가서
  
  length = len(content)
  
  df = pd.DataFrame(content)
  
  df.index = (range(2, length+2))
  df.loc[0, :] = ['temp']
  df.loc[1, :] = ['temp']
  df.loc[length+2, :] = ['temp']
  df = df.sort_index(axis=0)
  df['frame_index']= df.index
  
  result_df = pd.DataFrame(columns=['frame_index', 'acc', 'h1', 'h2', 'w1', 'w2'])
  
  num = 0
  for i in range(0, len(df)):
    content = df.iloc[i, 0]
    if 'person: ' in content:
      split_by_person = content.split('person: ')
      num_of_person = len(split_by_person)
      numbers = re.compile('[0-9]+')
      frame_index = df.iloc[i, 1]
      
      for j in range(1, num_of_person):
        c = list(map(int, numbers.findall(split_by_person[j])))
        acc = int(split_by_person[j].split('%')[0])
        result_df.loc[num, :] = [frame_index, acc, c[2], c[2]+c[4], c[1], c[1]+c[3]]
        num+=1
  
  result_df = result_df.loc[result_df['acc']>=accuracy, :]
  result_df.index = range(0, len(result_df))
  
  return result_df
  
# 사람이 나온 프레임 추출
def person_extraction(video_dir, df) :
  capture = cv2.VideoCapture(video_dir)
  
  i=0
  num = 0 # 파일 명의 인덱스를 만들어 주기 위한 변수
  
  while num < len(df)-1:
    ret, frame = capture.read()
    
    while True:
      
      if i == df.iloc[num, 0]:
        h1=int(df.iloc[num, 2])
        h2=int(df.iloc[num, 3])
        w1=int(df.iloc[num, 4])
        w2=int(df.iloc[num, 5])
        person = image_cropping(frame, h1, h2, w1, w2)
        person_half = person[0:len(person)//2, :]
        cv2.imwrite(f'/content/gdrive/My Drive/darknet/data/result/person/person{num}.png', person)
        cv2.imwrite(f'/content/gdrive/My Drive/darknet/data/result/person_half/person_half{num}.png', person_half)
        num+=1
        
      else : break
    i+=1

def face_detect(num):
  
  face_cascade = cv2.CascadeClassifier('/content/gdrive/My Drive/darknet/data/face_detect/data/haarcascades/haarcascade_frontalface_default.xml')
  image_list = [f'/content/gdrive/My Drive/darknet/data/result/person_half/person_half{i}.png' for i in range(0, num)]
  imgNum = 0
  i=0
  
  while True :
    try :
      image=cv2.imread(image_list[i])
      grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      faces = face_cascade.detectMultiScale(grayImage, 1.03, 5)
      
      for (x,y,w,h) in faces:
        if (y - int(h / 4)<0)|(y + h + int(h / 4)<0)|(x - int(w / 4)<0)|(x + w + int(w / 4)<0):break
        # cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        cropped = image[y - int(h / 4):y + h + int(h / 4), x - int(w / 4):x + w + int(w / 4)]
        # 이미지를 저장
        height, width, channel = cropped.shape
        img_result = cv2.pyrUp(cropped, (8*width, 8*height), borderType=cv2.BORDER_DEFAULT)
        cv2.imwrite(f"/content/gdrive/My Drive/darknet/data/result/face/face{imgNum}.png", img_result) # cropped , img_result
        imgNum += 1
    
    except : pass
    
    i+=1
    if i==num : break
  
def image_cropping(image, h1, h2, w1, w2):
  return image[h1:h2, w1:w2]

### 3-3 (함수) Person profile

In [0]:
# gender, age, emotion, landmark code
def renderResultOnImage( result, img ):
  
  # landmark
  for currFace in result:
    faceRectangle = currFace['faceRectangle']
    cv2.rectangle( img,(faceRectangle['left'],faceRectangle['top']),
                       (faceRectangle['left']+faceRectangle['width'], faceRectangle['top'] + faceRectangle['height']),
                  color = (255,0,0), thickness = 2 )
    
    faceLandmarks = currFace['faceLandmarks']
    
    for _, currLandmark in faceLandmarks.items():
      cv2.circle( img, (int(currLandmark['x']),int(currLandmark['y'])), color = (0,255,0), thickness= -1, radius = 1 )
  
  # gender, age, emotion
  # 여기서 혹시 더 추가하고 싶은거 있으면 face api 홈페이지 들어가서 parameter 보고 필요한 거 있으면 추가하도록
  for currFace in result:
    faceRectangle = currFace['faceRectangle']
    faceAttributes = currFace['faceAttributes']
    
    textToWrite = "%c (%d)" % ( 'M' if faceAttributes['gender']=='male' else 'F', faceAttributes['age'])
    # textToWrite2 = '%s'  % (faceAttributes['emotion'])
    cv2.putText( img, textToWrite, (faceRectangle['left'],faceRectangle['top']-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1 )
    # cv2.putText( img, textToWrite2, (faceRectangle['width'],faceRectangle['top']-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 3 )

### 1-3 YOLO 영상 및 좌표 저장

In [0]:
!ls -la '/content/gdrive/My Drive/darknet/data/data/'

In [0]:
# 좌표 코드
!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -ext_output -dont_show /content/gdrive/My\ Drive/darknet/data/data/video_input.mp4 -i 0 -out_filename /content/gdrive/My\ Drive/darknet/data/data/video_yolo.mp4 | tee /content/gdrive/My\ Drive/darknet/data/data/video_yolo_log.txt

# 기존 코드
# !./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights  -dont_show data/a000_4.mp4 -i 0 -out_filename a000_4.mp4
# 영상을 여기에 띄우지말고 darknet -> test_yolo.mp4 생성되는 파일을 다운 받아서 확인.

### 1-4 동영상 설정시 유의사항
1. don't show 옵션을 설정하고 동영상은 다운로드 후 시청을 추천함.
   이유는 저장 및 환경에 엄청 예민하기 때문이다.

In [0]:
!ls -la '/content/gdrive/My Drive/darknet/data/data/'

In [0]:
log_dir = '/content/gdrive/My Drive/darknet/data/data/video_yolo_log.txt'
over_100_df = parsing_log(log_dir, 100) # 정확도 100 이상인 프레임만 추출
over_100_df.tail()

In [0]:
# # 내컴퓨터로 다운로드 코드 넣기.
# files.download('/content/gdrive/My Drive/darknet/data/data/video_yolo_log.txt')

## 2_Log를 활용한 추출

In [0]:
# 처음에 생성한 빈 폴더.
# os.chdir("/content/gdrive/My Drive/darknet/data/result/person/")
!ls -la '/content/gdrive/My Drive/darknet/data/result/person/'

In [0]:
video_dir = '/content/gdrive/My Drive/darknet/data/data/video_input.mp4'

person_extraction(video_dir, over_100_df) # YOLO가 포착한 좌표를 기준으로 사람이 있는 부분만 이미지로 추출

face_detect(len(over_100_df)) # 사람 이미지에서 얼굴을 포착하여 이미지로 추출

In [0]:
# 생성중인 person 폴더
!ls -la '/content/gdrive/My Drive/darknet/data/result/person/'

In [0]:
# 생성중인 face 폴더
# os.chdir("/content/gdrive/My Drive/darknet/data/result/face/")
!ls -la '/content/gdrive/My Drive/darknet/data/result/face/'

### 2-2 face image copy

In [0]:
!cp /content/gdrive/My\ Drive/darknet/data/result/face/face90.png /content/gdrive/My\ Drive/darknet/data/3DFaceReconstruction/input/

## 3_Face detection (person profile)
폴더 내에 있는 페이스 아이디 불러오기

In [0]:
# img 폴도 path 설정 이미지 파일있는 폴더로 face id 추출
cur_dir = '/content/gdrive/My Drive/darknet/data/3DFaceReconstruction/input/'
test_file = os.listdir(cur_dir)

test_faceId = []
file_name = [] 

for i in range(0, len(test_file)):
  file_name.append(cur_dir + test_file[i])
  
  with open (file_name[i], 'rb') as f:
    data = f.read()

  try:
    result = detectionRequest( json, data, headers, params )
    test_faceId.append( result[0]['faceId'])
    time.sleep(1)
    print("")
  except:
    print("filename: %s" %file_name[i])

In [0]:
# 원래 이미지의 고유 ID 코드가 나오는데 간혹 안나오는 경우도 있음
test_faceId

### 3-4 face & profile

In [0]:
# 얼굴 인식 파라미터
# returnFaceAttribites, returnFaceLandmarks (age, gender, emotion, landmarks)
params = { 'returnFaceAttributes': 'age,gender,emotion', 
      'returnFaceLandmarks': 'true'} 

# 건들지 마시게!
headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

# 이미지 시각화를 위한 path 설정 (개별)
pathToFileInDisk = '/content/gdrive/My Drive/darknet/data/3DFaceReconstruction/input/face90.png'
with open( pathToFileInDisk, 'rb' ) as f:
  data = f.read()


result = detectionRequest( json, data, headers, params )

# 시각화
if result is not None:
  
  data8uint = np.fromstring( data, np.uint8 ) 
  img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )

  renderResultOnImage( result, img )

  ig, ax = plt.subplots(figsize=(20, 20))
  ax.imshow( img )
  
result[0]['faceId']

In [0]:
result

In [0]:
# 좌표 저장
# eyeLeftInner | eyeRightInner | noseTip | mouthLeft | mouthRight
# pupilLeft | pupilRight

facial_list = ['pupilLeft', 'pupilRight', 'noseTip', 'mouthLeft', 'mouthRight']

col = ['x','y']
result1 = result[0]
result2 = result1['faceLandmarks']
result3 = [result2.get(part) for part in facial_list]
result4 = [i.values() for i in result3]
result5 = pd.DataFrame(result4,columns=col,index=['pupilLeft','pupilRight','noseTip','mouthLeft','mouthRight'])
np.savetxt('/content/gdrive/My Drive/darknet/data/3DFaceReconstruction/input/face90.txt', result5, fmt='%d')
result5

## 4_2D를 3D로 몽타주

In [0]:
os.chdir("/content/gdrive/My Drive/darknet/data/3DFaceReconstruction/")

In [0]:
!python demo.py

In [0]:
!ls -la '/content/gdrive/My Drive/darknet/data/3DFaceReconstruction/output/input'

In [0]:
import ctypes
import os
import sys

sys.path.append('..')

import pyglet
from pyglet.gl import *

from pywavefront import visualization
import pywavefront

# Create absolute path from this module
file_abspath = 'face90_mesh.obj'

rotation = 0
meshes = pywavefront.Wavefront(file_abspath)
window = pyglet.window.Window()
lightfv = ctypes.c_float * 4


@window.event
def on_resize(width, height):
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(60., float(width)/height, 1., 100.)
    glMatrixMode(GL_MODELVIEW)
    return True


@window.event
def on_draw():
    window.clear()
    glLoadIdentity()

    glLightfv(GL_LIGHT0, GL_POSITION, lightfv(-1.0, 1.0, 1.0, 0.0))
    glEnable(GL_LIGHT0)

    glTranslated(0.0, 0.0, -3.0)
    glRotatef(rotation, 0.0, 1.0, 0.0)
    glRotatef(0.0, 1.0, 0.0, 0.0)
    glRotatef(0.0, 0.0, 0.0, 0.0)

    glEnable(GL_LIGHTING)

    visualization.draw(meshes)


def update(dt): # 0~90도, 270~360도.
    global rotation
    rotation += 90.0 * dt

    if rotation > 720.0:
        rotation = 0.0

        

pyglet.clock.schedule(update)
pyglet.app.run()